In [1]:
#%reset -f
%matplotlib inline
import os
import sys
import pandas as pd
import numpy as np
import feather
import time

### Import Data Frame and create raw X and y arrays

In [2]:
t0 = time.time()
if not os.path.exists("ipums.feather"):
    if not os.path.exists("ipums.csv"):
        !gunzip -c ipums.csv.gz > ipums.csv
    !R -f ./ipums_feather.R 
df = feather.read_dataframe("ipums.feather")
t1 = time.time()
print("Time to read data via feather: %r" % (t1-t0))

Time to read data via feather: 5.145771741867065


In [3]:
target = df.columns[-1] ## last column is the response
cols = [c for c in df.columns if c != target]

In [4]:
X = np.array(df.ix[:,cols], dtype='float64', order='C')
y = np.array(df[target].values, dtype='float64')
print(X.shape)
print(y.shape)

(55776, 9732)
(55776,)


In [5]:
df = None ## free mem
import gc
gc.collect()

38

## TASK:
* Elastic Net Regression for Gaussian distribution
* predict last column INCEARN from all other columns
* alpha=0.5 (or ideally, 8 different values)
* lambda search (100 lambdas)
* Perform 5-fold cross-validation
* Compute validation RMSE
* Note: assume the dataset is dense, even though it isn't in this instance

### Vowpal Wabbit

In [ ]:
if not os.path.exists("train.vw"):
    vw = np.concatenate([y.reshape(y.shape[0],1),X], axis=1)
    np.savetxt("train.vw", vw, delimiter=" ", fmt="%g")
    !sed -i -e 's/ / |/' train.vw

In [ ]:
t0 = time.time()
## performs OOB validation
!./vw-8.20170116 -d train.vw 2>&1 | tee log.vw #--l1 1 --l2 1 --ftrl --passes 10 --cache_file cache.vw
t1 = time.time()
print("Time to run one model through Vowpal Wabbit: %r" % (t1-t0))

In [ ]:
print("RMSE")
## TODO - check whether 'average loss' is the right info
!echo "sqrt(`grep "average loss" log.vw | awk '{print $4}'`)" | bc -l

### Split data into train/valid

In [7]:
H = (int)(0.8*X.shape[0])
print(H)

44620


In [8]:
trainX = X[:H,:]
trainY = y[:H]
validX = X[H:,:]
validY = y[H:]
X = None
y = None

In [15]:
alphas = [r/7. for r in range(0,8)] ## final requirement for demo
#alphas = [0.5] ## faster for testing
print(alphas)

[0.0, 0.14285714285714285, 0.2857142857142857, 0.42857142857142855, 0.5714285714285714, 0.7142857142857143, 0.8571428571428571, 1.0]


### Scikit-Learn

In [ ]:
## TODO

### TensorFlow

In [ ]:
import tensorflow as tf
from tensorflow.python.framework import ops

ops.reset_default_graph()
# Create graph
sess = tf.Session()

# Declare batch size
batch_size = 32

#Declare epochs
epochs = 1

# Initialize placeholders
x_data = tf.placeholder(shape=[None, trainX.shape[1]], dtype=tf.float32)
y_target = tf.placeholder(shape=[None, 1], dtype=tf.float32)

# Create variables for linear regression
A = tf.Variable(tf.random_normal(shape=[trainX.shape[1],1], seed = 42), name = "A")
b = tf.Variable(np.mean(trainY, dtype = np.float32), name = "b")

# Declare model operations
model_output = tf.add(tf.matmul(x_data, A), b)

# Declare the elastic net loss function
elastic_param1 = tf.placeholder(tf.float32, shape=None, name="e1")
elastic_param2 = tf.placeholder(tf.float32, shape=None, name="e2")
lambda_ = tf.placeholder(tf.float32, shape=None, name="lambda")
l1_a_loss = tf.reduce_mean(tf.abs(A))
l2_a_loss = tf.reduce_mean(tf.square(A))
e1_term = tf.multiply(elastic_param1, l1_a_loss)
e2_term = tf.multiply(elastic_param2, l2_a_loss)
loss = tf.expand_dims(
    tf.add(tf.reduce_mean(tf.square(y_target - model_output)),tf.multiply(lambda_,(tf.add(e1_term,e2_term)))), 
    0)

# Declare optimizer
my_opt = tf.train.GradientDescentOptimizer(0.01)
train_step = my_opt.minimize(loss)

np.random.seed(42)
def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]
        
# Training loop
init = tf.global_variables_initializer()
sess.run(init)
loss_vec = []
lambdas = [5.282175]
#alphas = [0.666667]
t0 = time.time()
for l in np.sort(lambdas)[::-1]:
    for a in alphas:
        # Initialize variables
        #sess.run(init)
        print("Lambda:",l,"Alpha:",a)
        for i in range(epochs):
            indx = list(range(len(trainX)))
            np.random.shuffle(indx)
            
            for rand_index in batch(indx, batch_size):
                rand_x = trainX[rand_index,:]
                rand_y = np.transpose([trainY[rand_index]])
                feed_dict = {
                  x_data: rand_x, y_target: rand_y,
                  elastic_param1: a, elastic_param2: (1. - a) / 2.0, 
                  lambda_: l,
                }
                sess.run(train_step, feed_dict=feed_dict)
print("Time for run through all alphas: ", time.time() - t0)
from sklearn.metrics import mean_squared_error
preds = np.dot(validX, sess.run(A)) + sess.run(b)
print("RMSE:",np.sqrt(mean_squared_error(validY, preds)))

### GLMNet

In [10]:
import scipy, importlib, pprint, matplotlib.pyplot as plt, warnings
import sys
sys.path.insert(0, "./glmnet_python/lib")
from glmnet import glmnet
from glmnetPlot import glmnetPlot 
from glmnetPrint import glmnetPrint
from glmnetCoef import glmnetCoef
from glmnetPredict import glmnetPredict
from cvglmnet import cvglmnet
from cvglmnetCoef import cvglmnetCoef
from cvglmnetPlot import cvglmnetPlot
from cvglmnetPredict import cvglmnetPredict

In [11]:
trainXscipy = scipy.array(trainX, dtype='float64')
trainYscipy = scipy.array(trainY, dtype='float64')
validXscipy = scipy.array(validX, dtype='float64')
validYscipy = scipy.array(validY, dtype='float64')
trainX = None ## free mem
trainY = None
validX = None
validY = None
gc.collect()

0

In [12]:
def run_alpha(a):
    t0 = time.time()
    fit = cvglmnet(nfolds=5, x=trainXscipy, y=trainYscipy, family="gaussian", alpha=a, nlambda=100)
    t1 = time.time()
    results = pd.DataFrame(columns=('alpha','lambda','rmse'))
    print("Time to train glmnet: %r" % (t1-t0))
    for l in fit['lambdau']:
        glmpred = glmnetPredict(fit, validXscipy, ptype = 'response', s = scipy.float64([l])).reshape(-1)
        rmse=np.sqrt(np.mean(np.square(glmpred[0] - validYscipy)))
        print(str(l) + " " + str(rmse))
        results = results.append([{'alpha': a, 'lambda': l, 'rmse':rmse}])
    results.to_csv("results.glmnet." + str(a))

In [13]:
import multiprocessing as mp
pool = mp.Pool(8)
zip(*pool.map(run_alpha, alphas))

KeyboardInterrupt: 

In [ ]:
#python glmnet - buggy?
#if not os.path.exists("glmnet.cpu.txt"):
#    !cat results.glmnet.* | grep -v rmse | sed 's/,/ /g' | awk '{print $2, $3, $4}' > glmnet.cpu.txt

In [ ]:
#ipums.R glmnet
if not os.path.exists("glmnet.cpu.txt"):
    !cat results.glmnet.* | grep -v ^c | sed 's/,/ /g' | awk '{print $1, $2, $3}' > glmnet.cpu.txt

### Plot solutions

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import math

In [ ]:
#uncomment and refresh this cell for live updates
#!grep validRMSE ~/pogs/examples/cpp/me0.6eb76ab.txt | awk '{print $6, $14, $20}' | sort -r -k3 > me0.6eb76ab.txt
res = pd.read_csv("me0.6eb76ab.txt", sep=" ",header=None,names=['alpha','lambda','rmse'])
best = res.ix[res['rmse']==np.min(res['rmse']),:]
print(best)
plt.scatter(np.log10(res['lambda']), res['alpha'], c=res['rmse'], cmap='jet', vmin=28500, vmax=42500)
plt.colorbar()
plt.annotate('o', xy=(np.log10(best['lambda']),best['alpha']), fontsize=50, horizontalalignment='center', verticalalignment='center')

In [ ]:
res = pd.read_csv("glmnet.cpu.txt", sep=" ",header=None,names=['alpha','lambda','rmse'])
best = res.ix[res['rmse']==np.min(res['rmse']),:]
print(best)
plt.scatter(np.log10(res['lambda']), res['alpha'], c=res['rmse'], cmap='jet', vmin=28500, vmax=42500)
plt.colorbar()
plt.annotate('o', xy=(np.log10(best['lambda']),best['alpha']), fontsize=50, horizontalalignment='center', verticalalignment='center')

In [ ]:
#glmnetPlot(fit, xvar = 'lambda', label = True)

### H2O

In [30]:
h2o.cluster().shutdown()

H2O session _sid_8d1f closed.


In [32]:
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_101"; Java(TM) SE Runtime Environment (build 1.8.0_101-b13); Java HotSpot(TM) 64-Bit Server VM (build 25.101-b13, mixed mode)
  Starting server from /home/arno/.pyenv/versions/3.6.0/envs/h2oai/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpisjtm8fi
  JVM stdout: /tmp/tmpisjtm8fi/h2o_arno_started_from_python.out
  JVM stderr: /tmp/tmpisjtm8fi/h2o_arno_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,04 secs
H2O cluster version:,3.11.0.99999
H2O cluster version age:,1 hour and 5 minutes
H2O cluster name:,H2O_from_python_arno_zcvtp4
H2O cluster total nodes:,1
H2O cluster free memory:,26.67 Gb
H2O cluster total cores:,40
H2O cluster allowed cores:,40
H2O cluster status:,"accepting new members, healthy"
H2O connection url:,http://127.0.0.1:54321
H2O connection proxy:,None


In [33]:
t0 = time.time()
df_hex = h2o.import_file("ipums.csv")
t1 = time.time()
print("Time to parse with H2O: %r" % (t1-t0))

Parse progress: |█████████████████████████████████████████████████████████| 100%
Time to parse with H2O: 12.206703662872314


In [35]:
train_hex = df_hex[:H,:]
valid_hex = df_hex[H:,:]

In [36]:
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

def run_alpha(args):
    cols, a, target, train_hex_id, valid_hex_id = args
    print("alpha: ", a)
    h2oglm = H2OGeneralizedLinearEstimator(nfolds=5,family="gaussian", alpha=a, lambda_search=True)
    h2oglm.train(x=cols, y=target, training_frame=h2o.get_frame(train_hex_id))
    print("rmse: ", str(h2oglm.model_performance(h2o.get_frame(valid_hex_id)).rmse()))
    
train_hex.refresh()
valid_hex.refresh()
import multiprocessing as mp
pool = mp.Pool(8)
t0 = time.time()
work = ((cols, a, target, train_hex.frame_id, valid_hex.frame_id) for a in alphas)
pool.map(run_alpha, work)
t1 = time.time()
print("Time to train H2O: %r" % (t1-t0))

alpha: 
 0.0alpha: alpha: alpha:  alpha:  0.42857142857142855 alpha: 0.57142857142857140.14285714285714285
alpha:   
 0.71428571428571430.2857142857142857alpha: 0.8571428571428571
 


1.0
glm Model Build progress: |glm Model Build progress: |glm Model Build progress: |glm Model Build progress: |glm Model Build progress: |glm Model Build progress: |glm Model Build progress: |glm Model Build progress: |███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100%
███rmse:  31180.05771259923
███████████| 100%
██████| 100%█
█rmse:  31180.8918314204
█rmse:  31107.897823484684
████████| 100%
████| 100%
█████| 100%
rmse:  31079.638497346365
rmse:  31139.010876402226
rmse:  31144.755781090662
█████| 100%
rmse:  31119.359360224007
████████████████